# Imagenette2 Dataset - Custom DataLoader & Modified Pooling Layers
I worked on the assigned task to load the Imagenette2 dataset, create my own DataLoader, 
modify the pooling layers in the model, train it, and evaluate its performance.
Keeping prints to a minimum to make the output clean.


In [4]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_data = ImageFolder(root="/content/imagenette2/train", transform=transform)
val_data = ImageFolder(root="/content/imagenette2/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop (1 epoch for now)
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

print("Training completed.")


Training completed.


In [10]:
num_epochs = 10  # or any number you want

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        ...
    print(f"Epoch [{epoch+1}/{num_epochs}] completed.")


Epoch [1/10] completed.
Epoch [2/10] completed.
Epoch [3/10] completed.
Epoch [4/10] completed.
Epoch [5/10] completed.
Epoch [6/10] completed.
Epoch [7/10] completed.
Epoch [8/10] completed.
Epoch [9/10] completed.
Epoch [10/10] completed.


In [17]:
# Cell: training (only if you need to retrain)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 10
train_losses, train_accs = [], []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    correct = total = 0
    t0 = time.time()

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        if (i+1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Loss {loss.item():.4f}")

    epoch_loss /= len(train_loader)
    epoch_acc = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    print(f"Epoch {epoch+1} done in {time.time()-t0:.1f}s — Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%")

# Save the trained model
torch.save(model.state_dict(), model_path)
print("Saved model to", model_path)


Epoch 1/10, Batch 50/296, Loss 1.9761
Epoch 1/10, Batch 100/296, Loss 2.2101
Epoch 1/10, Batch 150/296, Loss 1.9431
Epoch 1/10, Batch 200/296, Loss 1.9894
Epoch 1/10, Batch 250/296, Loss 2.0154
Epoch 1 done in 33.3s — Loss: 2.0262, Acc: 27.50%
Epoch 2/10, Batch 50/296, Loss 1.7215
Epoch 2/10, Batch 100/296, Loss 1.6432
Epoch 2/10, Batch 150/296, Loss 1.6732
Epoch 2/10, Batch 200/296, Loss 1.8642
Epoch 2/10, Batch 250/296, Loss 1.6175
Epoch 2 done in 33.1s — Loss: 1.8361, Acc: 35.71%
Epoch 3/10, Batch 50/296, Loss 1.8451
Epoch 3/10, Batch 100/296, Loss 1.7683
Epoch 3/10, Batch 150/296, Loss 1.5949
Epoch 3/10, Batch 200/296, Loss 2.2609
Epoch 3/10, Batch 250/296, Loss 1.5402
Epoch 3 done in 33.4s — Loss: 1.7521, Acc: 39.29%
Epoch 4/10, Batch 50/296, Loss 1.7073
Epoch 4/10, Batch 100/296, Loss 1.8022
Epoch 4/10, Batch 150/296, Loss 1.7264
Epoch 4/10, Batch 200/296, Loss 1.7417
Epoch 4/10, Batch 250/296, Loss 1.9036
Epoch 4 done in 33.0s — Loss: 1.6791, Acc: 42.57%
Epoch 5/10, Batch 50/296

In [20]:
# Replace IMG_SIZE with whatever size you used in training
IMG_SIZE = 224  

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [25]:
train_dir = "/kaggle/input/imagenette2-320/imagenette2-320/train"
test_dir  = "/kaggle/input/imagenette2-320/imagenette2-320/val"


In [26]:
from torchvision import datasets, transforms

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(test_dir, transform=test_transform)

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [27]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim


In [28]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [29]:
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
val_data   = datasets.ImageFolder(test_dir, transform=val_transform)


In [30]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)


In [31]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_data.classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [33]:
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/3], Loss: 0.6323
Epoch [2/3], Loss: 0.3933
Epoch [3/3], Loss: 0.2935


In [36]:
# Number of epochs increased
num_epochs = 10  

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100.0 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%")

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100.0 * val_correct / val_total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/10], Loss: 0.2121, Train Accuracy: 93.38%
Validation Accuracy: 79.72%
Epoch [2/10], Loss: 0.1758, Train Accuracy: 94.10%
Validation Accuracy: 83.03%
Epoch [3/10], Loss: 0.1393, Train Accuracy: 95.43%
Validation Accuracy: 85.48%
Epoch [4/10], Loss: 0.1148, Train Accuracy: 96.34%
Validation Accuracy: 84.89%
Epoch [5/10], Loss: 0.1092, Train Accuracy: 96.54%
Validation Accuracy: 85.91%
Epoch [6/10], Loss: 0.0733, Train Accuracy: 97.67%
Validation Accuracy: 87.69%
Epoch [7/10], Loss: 0.0983, Train Accuracy: 96.90%
Validation Accuracy: 86.19%
Epoch [8/10], Loss: 0.0918, Train Accuracy: 97.18%
Validation Accuracy: 88.92%
Epoch [9/10], Loss: 0.0650, Train Accuracy: 97.82%
Validation Accuracy: 82.60%
Epoch [10/10], Loss: 0.0684, Train Accuracy: 97.86%
Validation Accuracy: 88.84%


In [37]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
